In [2]:
import os

In [3]:
%pwd

'c:\\Users\\PURUSHOTHAM P\\Desktop\\kidney-disease-classification\\research'

In [4]:
os.chdir("../")

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/purushothamp2003/kidney-disease-classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="purushothamp2003"
os.environ["MLFLOW_TRACKING_PASSWORD"]="Pruthvi@2003"

In [6]:
import tensorflow as tf 

In [7]:
file_path = "artifacts/training/model.h5"
print(os.path.exists(file_path)) 

True


In [8]:
# model = tf.keras.models.load_model("artifacts\training\model.h5")
file_path = "artifacts/training/model.h5"

# Check if the file exists
if os.path.exists(file_path):
    # Load the model
    model = tf.keras.models.load_model(file_path)
    print("Model loaded successfully.")
else:
    print("Error: File not found.")

Model loaded successfully.


In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/purushothamp2003/kidney-disease-classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config



In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [15]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-08-06 23:03:30,897: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-06 23:03:30,904: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-06 23:03:30,904: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 25s 3s/step - loss: 2.0731e-05 - accuracy: 1.0000
[2024-08-06 23:03:57,173: INFO: common: json file saved at: scores.json]
[2024-08-06 23:03:57,251: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)': /purushothamp2003/kidney-disease-classification.mlflow/api/2.0/mlflow/runs/create]


2024/08/06 23:03:59 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-08-06 23:04:00,305: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\PURUSH~1\AppData\Local\Temp\tmpt1mxe35q\model\data\model\assets
[2024-08-06 23:04:01,124: INFO: builder_impl: Assets written to: C:\Users\PURUSH~1\AppData\Local\Temp\tmpt1mxe35q\model\data\model\assets]
[2024-08-06 23:05:30,152: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)': /purushothamp2003/kidney-disease-classification.mlflow/api/2.0/mlflow-artifacts/artifacts/2355d1c903224547ab02fdcfe7f328ed/653ed07cc0c6495893e1f3dd902cd1f3/artif

Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/08/06 23:06:48 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
